In [1]:
import os
import scipy
import numpy as np
import matplotlib.pyplot as plt
import skimage

from skimage import exposure
from skimage import feature
from skimage import io
from skimage import color
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from sklearn import datasets
from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')

In [2]:
def show(img):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(img, cmap=plt.cm.gray)
    ax.set_axis_off()
    plt.show()

In [3]:
#selecionando as classes
MAIN_DIR = "frutas_dataset_train"
y = [name for name in os.listdir(MAIN_DIR) if os.path.isdir(os.path.join(MAIN_DIR, name))]
result = [os.path.join(dp, f) for dp, dn, filenames in os.walk(MAIN_DIR) for f in filenames if os.path.splitext(f)[1] == '.jpg']
#transformar as labels de string para ints
le = preprocessing.LabelEncoder()
encoded = le.fit_transform(y)

In [4]:
#qtde de arquivos por pasta
num_files = []
for i in y:
    DIR = "frutas_dataset_train/" + i + "/"
    num_files.append(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

X = np.zeros((len(result), 10)) #n_samples x n_features

In [5]:
#criando o target label == classes das frutas
k = []
for i in range(len(y)):
    for j in range(num_files[i]):
        k.append(encoded[i])

# Gerar a LBP das imagens

In [88]:
#criando a matriz geral
for i in range(len(result)):
    im = color.rgb2gray(io.imread(result[i]))
    lbp = feature.texture.local_binary_pattern(im, 8, 2, method='uniform')
    histogram = scipy.stats.itemfreq(lbp)
    x = histogram[:,1]
    norm_hist = normalize(x[:,np.newaxis], axis=0).ravel()
    X[i,:] = x

In [90]:
#salvando estado da matriz X e do vetor de target labels k
np.savetxt("extracted_features/x_lbp.txt", X, delimiter=";")
np.savetxt("extracted_features/k.txt", k, delimiter=";")

# Ler a LBP do txt

In [6]:
#recuperar estado
X_lbp = np.genfromtxt('extracted_features/x_lbp.txt', delimiter=';')
k = np.genfromtxt('extracted_features/k.txt', delimiter=';')

In [7]:
from sklearn.model_selection import train_test_split
X_lbp_train, X_lbp_test, k_lbp_train, k_lbp_test = train_test_split(X_lbp, k, test_size=0.5) #test size = 50%

In [8]:
knn_lbp = KNeighborsClassifier(n_neighbors=3)

In [9]:
knn_lbp.fit(X_lbp_train, k_lbp_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [10]:
k_lbp_pred = knn_lbp.predict(X_lbp_test)

In [11]:
from sklearn import metrics
print("Taxa de acerto:", metrics.accuracy_score(k_lbp_test, k_lbp_pred))

Taxa de acerto: 0.7867298578199052


In [12]:
def calcula_lbp_hist(im):
    lbp = feature.texture.local_binary_pattern(im, 8, 2, method='uniform')
    histogram = scipy.stats.itemfreq(lbp)
    x = histogram[:,1]
    return x

In [13]:
def get_class(encoded, y, num):
    for i in range(len(encoded)):
        if(encoded[i] == num):
            break
    return y[i]

In [14]:
im = color.rgb2gray(io.imread("cashew_039.jpg"))
x = calcula_lbp_hist(im)
a = knn_lbp.predict(x.reshape(1,-1))
print(get_class(encoded, y, a))

cashew


In [15]:
im = color.rgb2gray(io.imread("watermelon_063.jpg"))
x = calcula_lbp_hist(im)
a = knn_lbp.predict(x.reshape(1,-1))
print(get_class(encoded, y, a))

watermelon


# Gerar a matriz de confusão

In [16]:
from sklearn.metrics import confusion_matrix

k_pred = knn_lbp.predict(X_lbp_test)
conf_matrix = confusion_matrix(k_lbp_test, k_lbp_pred)
conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

In [17]:
np.savetxt("confusion_matrix_script/input_lbp.csv", conf_matrix, delimiter=';', fmt='%.3f')

# Gerar o Histograma de Cores das imagens

In [44]:
X_hist = np.zeros((len(result), 768))

for i in range(len(result)):
    im = skimage.img_as_float(io.imread(result[i]))
    hist_r = exposure.histogram(im[:,:,0])
    hist_g = exposure.histogram(im[:,:,1])
    hist_b = exposure.histogram(im[:,:,2])
    x = np.append(hist_r[0], hist_g[0])
    x = np.append(x, hist_b[0])
    X_hist[i,:] = x

In [60]:
np.savetxt("extracted_features/x_hist.txt", X_hist, delimiter=";")

# Ler o Histograma de Cor do txt

In [18]:
X_hist = np.genfromtxt('extracted_features/x_hist.txt', delimiter=';')
k = np.genfromtxt('extracted_features/k.txt', delimiter=';')

In [19]:
X_hist_train, X_hist_test, k_hist_train, k_hist_test = train_test_split(X_hist, k, test_size=0.5)

In [20]:
knn_hist = KNeighborsClassifier(n_neighbors=3)

In [21]:
knn_hist.fit(X_hist_train, k_hist_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [22]:
k_hist_pred = knn_hist.predict(X_hist_test)

In [23]:
print("Taxa de acerto:", metrics.accuracy_score(k_hist_test, k_hist_pred))

Taxa de acerto: 0.6034755134281201


In [24]:
def calcula_hist(im):
    hist_r = exposure.histogram(im[:,:,0])
    hist_g = exposure.histogram(im[:,:,1])
    hist_b = exposure.histogram(im[:,:,2])
    x = np.append(hist_r[0], hist_g[0])
    x = np.append(x, hist_b[0])
    return x

In [25]:
im = skimage.img_as_float(io.imread("cashew_039.jpg"))
x = calcula_hist(im)
a = knn_hist.predict(x.reshape(1,-1))
print(get_class(encoded, y, a))

cashew


In [26]:
im = skimage.img_as_float(io.imread("watermelon_063.jpg"))
x = calcula_hist(im)
a = knn_hist.predict(x.reshape(1,-1))
print(get_class(encoded, y, a))

watermelon


In [27]:
im = skimage.img_as_float(io.imread("fuji.gif"))
x = calcula_hist(im)
a = knn_hist.predict(x.reshape(1,-1))
print(get_class(encoded, y, a))

orange


# Gerar a matriz de confusão

In [28]:
conf_matrix_hist = confusion_matrix(k_hist_test, k_hist_pred)
conf_matrix_hist = conf_matrix_hist.astype('float') / conf_matrix_hist.sum(axis=1)[:, np.newaxis]

In [29]:
np.savetxt("confusion_matrix_script/input_hist.csv", conf_matrix_hist, delimiter=';', fmt='%.3f')

# Haralick Features

In [119]:
X_har = np.zeros((len(result), 20))
for i in range(len(result)):
    x = []
    im = skimage.img_as_ubyte(color.rgb2gray(io.imread(result[i])))
    f = feature.texture.greycomatrix(im, [1,2], [0, np.pi/2], 256, symmetric=True, normed=True)
    x.append(feature.texture.greycoprops(f, prop = 'contrast')[0,0])
    x.append(feature.texture.greycoprops(f, prop = 'contrast')[0,1])
    x.append(feature.texture.greycoprops(f, prop = 'contrast')[1,0])
    x.append(feature.texture.greycoprops(f, prop = 'contrast')[1,1])
#     x.append(feature.texture.greycoprops(f, prop = 'contrast')[0,2])
#     x.append(feature.texture.greycoprops(f, prop = 'contrast')[0,3])
    x.append(feature.texture.greycoprops(f, prop = 'dissimilarity')[0,0])
    x.append(feature.texture.greycoprops(f, prop = 'dissimilarity')[0,1])
    x.append(feature.texture.greycoprops(f, prop = 'dissimilarity')[1,0])
    x.append(feature.texture.greycoprops(f, prop = 'dissimilarity')[1,1])
#     x.append(feature.texture.greycoprops(f, prop = 'dissimilarity')[0,2])
#     x.append(feature.texture.greycoprops(f, prop = 'dissimilarity')[0,3])
    x.append(feature.texture.greycoprops(f, prop = 'homogeneity')[0,0])
    x.append(feature.texture.greycoprops(f, prop = 'homogeneity')[0,1])
    x.append(feature.texture.greycoprops(f, prop = 'homogeneity')[1,0])
    x.append(feature.texture.greycoprops(f, prop = 'homogeneity')[1,1])
#     x.append(feature.texture.greycoprops(f, prop = 'homogeneity')[0,2])
#     x.append(feature.texture.greycoprops(f, prop = 'homogeneity')[0,3])
    x.append(feature.texture.greycoprops(f, prop = 'ASM')[0,0])
    x.append(feature.texture.greycoprops(f, prop = 'ASM')[0,1])
    x.append(feature.texture.greycoprops(f, prop = 'ASM')[1,0])
    x.append(feature.texture.greycoprops(f, prop = 'ASM')[1,1])
#     x.append(feature.texture.greycoprops(f, prop = 'ASM')[0,2])
#     x.append(feature.texture.greycoprops(f, prop = 'ASM')[0,3])
    x.append(feature.texture.greycoprops(f, prop = 'energy')[0,0])
    x.append(feature.texture.greycoprops(f, prop = 'energy')[0,1])
    x.append(feature.texture.greycoprops(f, prop = 'energy')[1,0])
    x.append(feature.texture.greycoprops(f, prop = 'energy')[1,1])
#     x.append(feature.texture.greycoprops(f, prop = 'energy')[0,2])
#     x.append(feature.texture.greycoprops(f, prop = 'energy')[0,3])
    X_har[i,:] = x 

In [ ]:
np.savetxt("extracted_features/x_har.txt", X_har, delimiter=';')

# Ler do txt

In [32]:
X_har = np.genfromtxt("extracted_features/x_har.txt", delimiter=';')

In [130]:
k = np.genfromtxt('extracted_features/k.txt', delimiter=';')

In [131]:
from sklearn.model_selection import train_test_split
X_har_train, X_har_test, k_har_train, k_har_test = train_test_split(X_har, k, test_size=0.5)

In [132]:
knn_har = KNeighborsClassifier(n_neighbors=3)

In [133]:
knn_har.fit(X_har_train, k_har_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [134]:
k_har_pred = knn_har.predict(X_har_test)

In [135]:
from sklearn import metrics
print("Taxa de acerto:", metrics.accuracy_score(k_har_test, k_har_pred))

Taxa de acerto: 0.5521327014218009
